In [1]:
import os
import glob
import math
import pickle

import numpy as np
import pandas as pd
import geopandas as gpd
import torch

# import network_io as nio
import processing_io as pio

# Parameters to define
is_for_1pm = True
result_path = "results/dataset_1pm_0-1400_more_features"

if is_for_1pm:
    base_dir = '../../../../data/pop_1pm_with_policies/'
    subdirs_pattern = os.path.join(base_dir, 'output_networks_*')
    subdirs = list(set(glob.glob(subdirs_pattern)))
    subdirs.sort()
    base_output_links_no_policies = '../../../data/pop_1pm/output_1pm/'
else:
    base_path = '../../../../data/pop_1pct_with_policies/output_networks_100/'
    network_folders = glob.glob(os.path.join(base_path, 'network_d_*'))
    base_output_links_no_policies = '../../../../data/pop_1pct/output_1pct/'
    
gdf_basecase_mean = gpd.read_file('results/gdf_basecase_mean_pop_1pm.geojson')

## Process results

Process the outputs of the simulations for further usage by GNN.

In [2]:
# Read all network data into a dictionary of GeoDataFrames
result_dic = {}
result_dic_single_districts = {}
base_network_no_policies = None

if is_for_1pm:
    # Loop through each subdirectory
    for subdir in subdirs:
        print(f'Accessing folder: {subdir}')
        print(len(os.listdir(subdir)))
        for network in os.listdir(subdir):
            folder = os.path.join(subdir, network)
            file_path = os.path.join(subdir, network)
            if file_path.endswith(".DS_Store"):
                continue
            policy_key = pio.create_policy_key_1pm(network)
            gdf = pio.read_output_links(file_path)
            if (gdf is not None):
                gdf_extended = pio.extend_geodataframe(gdf_base=gdf_basecase_mean, gdf_to_extend=gdf, column_to_extend='highway', new_column_name='highway')
                gdf_extended = pio.extend_geodataframe(gdf_base=gdf_basecase_mean, gdf_to_extend=gdf_extended, column_to_extend='vol_car', new_column_name='vol_car_base_case')
                result_dic[policy_key] = gdf_extended
    base_network_no_policies = gdf_basecase_mean
else:
    for folder in network_folders:
        policy_key = pio.create_policy_key_1pct(os.path.basename(folder))
        try:
            gdf = pio.read_output_links(folder)
        except Exception:
            print(f"Error reading folder: {folder}")
        
        if gdf is not None:
            result_dic[policy_key] = gdf
        is_single  = pio.is_single_district(os.path.basename(folder))
        if is_single:
            result_dic_single_districts[policy_key] = gdf
        break
        
result_dic["base_network_no_policies"] = base_network_no_policies

Accessing folder: ../../../../data/pop_1pm_with_policies/output_networks_100
101
Accessing folder: ../../../../data/pop_1pm_with_policies/output_networks_1000
100
Accessing folder: ../../../../data/pop_1pm_with_policies/output_networks_1100
100
Accessing folder: ../../../../data/pop_1pm_with_policies/output_networks_1200
100
Accessing folder: ../../../../data/pop_1pm_with_policies/output_networks_1300
100
Accessing folder: ../../../../data/pop_1pm_with_policies/output_networks_1400
100
Accessing folder: ../../../../data/pop_1pm_with_policies/output_networks_200
101
Accessing folder: ../../../../data/pop_1pm_with_policies/output_networks_300
100
Accessing folder: ../../../../data/pop_1pm_with_policies/output_networks_400
100
Accessing folder: ../../../../data/pop_1pm_with_policies/output_networks_500
100
Accessing folder: ../../../../data/pop_1pm_with_policies/output_networks_600
100
Accessing folder: ../../../../data/pop_1pm_with_policies/output_networks_700
100
Accessing folder: ../..

## Analyze results and plot

In [3]:
# pio.analyze_geodataframes(result_dic=result_dic, consider_only_highway_edges=True)

In [4]:
# pio.analyze_geodataframes(result_dic=result_dic, consider_only_highway_edges=False)

In [5]:
# for policy_key, gdf in result_dic.items():
#     if policy_key != "base_network_no_policies":
#         gdf_extended = pio.extend_geodataframe(gdf_base=gdf_basecase_mean, gdf_to_extend=gdf, column_to_extend='vol_car', new_column_name='vol_car_base_case')
#         result_dic[policy_key] = gdf_extended

## Save for further processing with GNN

In [6]:
data_processed = pio.process_result_dic(result_dic)
# data_processed_single_districts = pio.process_result_dic(result_dic_single_districts)

torch.save(data_processed, result_path + '.pt')
# torch.save(data_processed_single_districts, result_path + '_single_districts.pt')

In [ ]:
# torch.set_printoptions(precision=4, sci_mode=False)